<a href="https://colab.research.google.com/github/talhaanwarch/ODIR2019/blob/master/focal_loss_custom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

#connect drive with colab notebook/

from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


https://github.com/keras-team/keras/issues/8130
https://stackoverflow.com/questions/56796331/how-to-input-multiple-images-with-flow-from-dataframe-in-keras

In [2]:
#following are libraries to unpack 7z file
!pip install patool 
!pip install pyunpack
from pyunpack import Archive

     |████████████████████████████████| 81kB 3.3MB/s 
  Created wheel for pyunpack: filename=pyunpack-0.1.2-cp36-none-any.whl size=5162 sha256=0803d90f2946c6cec14faad47e9c1496113e2ddd9a2dbd1e7f16a219c019b9fb
  Stored in directory: /root/.cache/pip/wheels/af/44/08/60613970881e542c0baad1f2dea5ed8e6716bc573f49197b7e
Successfully built pyunpack


In [0]:
import pandas as pd
import numpy as np
import cv2

In [0]:

#create a folder name as data
!mkdir data

In [0]:
#path where our data is saved
path='/content/drive/My Drive/ghina/thesis/'

In [0]:
#extract the 7z file in data folder we created above
#this not only extract the data but move it from drive to colab for efficinet processing
#it will take some time
Archive(path+'ODIR-5K_Training_Images.7z').extractall('data')

In [0]:
#read the annottion file
annotation=pd.read_excel(path+'ODIR-5K_Training_Annotations(Updated)_V2.xlsx')

In [0]:

#normal (N), diabetes (D), glaucoma (G), cataract (C), AMD (A), hypertension (H), myopia (M) and other diseases/abnormalities (O)

In [9]:
#print first 5 rows of annotation file
annotation=annotation.sample(frac=1)
annotation.head()

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O
2577,3200,57,Male,3200_left.jpg,3200_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0
3088,4229,64,Male,4229_left.jpg,4229_right.jpg,normal fundus,moderate non proliferative retinopathy,0,1,0,0,0,0,0,0
1344,1519,68,Female,1519_left.jpg,1519_right.jpg,normal fundus,pathological myopia,0,0,0,0,0,0,1,0
468,469,69,Female,469_left.jpg,469_right.jpg,moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,0
1371,1551,50,Male,1551_left.jpg,1551_right.jpg,pathological myopia,pathological myopia,0,0,0,0,0,0,1,0


In [0]:
train_annotation=annotation[0:2500]
val_annotation=annotation[2500:3500]


In [0]:
from keras_preprocessing.image import ImageDataGenerator

train_data_gen= ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=.1,
    rescale=1/255,
    fill_mode='nearest'
)

val_data_gen= ImageDataGenerator(
    rescale=1/255
    
)

In [0]:

def get_flow_from_dataframe(generator, dataframe,directory_path,
                            image_shape=(224, 224),
                            #subset='training',
                             batch_size=50):
    train_generator_1 = generator.flow_from_dataframe(dataframe,
                                                      directory=directory_path,
                                                      target_size=image_shape,
                                                      x_col="Left-Fundus",
                                                      y_col=['N'	,'D'	,'G',	'C',	'A'	,'H'	,'M',	'O'],
                                                      class_mode='other',
                                                      shuffle=False,
                                                      batch_size=batch_size,
                                                      seed=7,
                                                      #subset=subset
                                                      )

    train_generator_2 = generator.flow_from_dataframe(dataframe, 
                                                      directory=directory_path,
                                                      target_size=image_shape,
                                                      x_col="Right-Fundus",
                                                      y_col=['N'	,'D'	,'G',	'C',	'A'	,'H'	,'M',	'O'],
                                                      class_mode='other',
                                                      shuffle=False,
                                                      batch_size=batch_size,
                                                      seed=7,
                                                      #subset=subset
                                                      )
    while True:
        x_1 = train_generator_1.next()
        x_2 = train_generator_2.next()

        yield [x_1[0], x_2[0]], x_1[1]

In [0]:
batch_size=50
train_gen = get_flow_from_dataframe(train_data_gen, train_annotation, image_shape=(224, 224),
                                        directory_path="data/ODIR-5K_Training_Dataset/",batch_size=batch_size)
valid_gen = get_flow_from_dataframe(val_data_gen, val_annotation, image_shape=(224, 224),
                                        directory_path="data/ODIR-5K_Training_Dataset/",batch_size=batch_size)

In [14]:
from keras.models import Sequential,Model
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPool2D, BatchNormalization,Input,MaxPooling2D,GlobalMaxPooling2D,concatenate
from keras.layers import GlobalAveragePooling2D
from keras.applications.resnet50 import ResNet50
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

Using TensorFlow backend.


In [0]:
inp1= Input(shape=(224, 224, 3))
inp2= Input(shape=(224, 224, 3))

X1=BatchNormalization()(inp1)
X1=Conv2D(32, (3, 3), padding='same',activation='relu')(X1)
X1=Conv2D(32, (3, 3), padding='same',activation='relu')(X1)
X1=MaxPooling2D(pool_size=(2, 2))(X1)
X1=Conv2D(32, (3, 3), padding='same',activation='relu')(X1)
X1=Conv2D(32, (3, 3), padding='same',activation='relu')(X1)
X1=GlobalMaxPooling2D()(X1)


X2=BatchNormalization()(inp2)
X2=Conv2D(32, (3, 3), padding='same',activation='relu')(X2)
X2=Conv2D(32, (3, 3), padding='same',activation='relu')(X2)
X2=MaxPooling2D(pool_size=(2, 2))(X2)
X2=Conv2D(32, (3, 3), padding='same',activation='relu')(X2)
X2=Conv2D(32, (3, 3), padding='same',activation='relu')(X2)
X2=GlobalMaxPooling2D()(X2)

X=concatenate([X1,X2])
X=Dropout(0.25)(X)
X=Dense(512,activation='relu')(X)
X=Dropout(0.25)(X)
out=Dense(8, activation='sigmoid')(X)

model=Model(inputs=[inp1,inp2],outputs=out)



In [0]:
import keras.backend as K
def focal_loss(y_true, y_pred, alpha = 0.75, gamma = 2):
  alpha_factor = K.ones_like(y_true) * alpha
  alpha_factor = tf.where(K.equal(y_true, 1), alpha_factor, 1 - alpha_factor)
  focal_weight = tf.where(K.equal(y_true, 1), 1 - y_pred, y_pred)
  focal_weight = alpha_factor * focal_weight ** gamma
  loss = focal_weight * K.binary_crossentropy(y_true, y_pred)
  return loss

In [0]:
model.compile('Adadelta',loss=focal_loss,metrics=['accuracy'])

In [0]:
step_size_train=2500/50
step_size_val=1000/50


In [20]:
model.fit_generator(train_gen, epochs=5,
                        steps_per_epoch=step_size_train,
                        use_multiprocessing=True,
                        validation_data=valid_gen,
                        validation_steps=step_size_val,
                        verbose=2
                    
                        )

Epoch 1/5
Found 1000 validated image filenames.
Found 2500 validated image filenames.
Found 1000 validated image filenames.
Found 2500 validated image filenames.
 - 416s - loss: 0.0469 - acc: 0.2856 - val_loss: 0.0434 - val_acc: 0.2640
Epoch 2/5
 - 366s - loss: 0.0449 - acc: 0.3124 - val_loss: 0.0430 - val_acc: 0.3430
Epoch 3/5
 - 365s - loss: 0.0446 - acc: 0.3260 - val_loss: 0.0428 - val_acc: 0.3480
Epoch 4/5
 - 364s - loss: 0.0443 - acc: 0.3220 - val_loss: 0.0427 - val_acc: 0.3630
Epoch 5/5
 - 364s - loss: 0.0441 - acc: 0.3244 - val_loss: 0.0425 - val_acc: 0.3670


In [21]:
y_pred = model.predict_generator(valid_gen,steps=step_size_val)


Found 1000 validated image filenames.
Found 1000 validated image filenames.


In [0]:
y_pred1=y_pred.copy()
y_pred1[y_pred1>=0.35] = 1
y_pred1[y_pred1<0.35] = 0
y_pred1=y_pred1.astype(int)

In [35]:
y_true=val_annotation.iloc[:,-8::].values
y_true.shape

(1000, 8)

In [36]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import multilabel_confusion_matrix

multilabel_confusion_matrix(y_true, y_pred1)

array([[[  0, 678],
        [  0, 322]],

       [[  0, 689],
        [  0, 311]],

       [[195, 749],
        [  5,  51]],

       [[420, 521],
        [ 14,  45]],

       [[376, 587],
        [ 17,  20]],

       [[942,  31],
        [ 27,   0]],

       [[634, 320],
        [ 26,  20]],

       [[  0, 697],
        [  0, 303]]])

In [39]:
target_names = ['N'	,'D'	,'G',	'C',	'A'	,'H'	,'M',	'O']
print(classification_report(y_true, y_pred1, target_names=target_names))

              precision    recall  f1-score   support

           N       0.32      1.00      0.49       322
           D       0.31      1.00      0.47       311
           G       0.06      0.91      0.12        56
           C       0.08      0.76      0.14        59
           A       0.03      0.54      0.06        37
           H       0.00      0.00      0.00        27
           M       0.06      0.43      0.10        46
           O       0.30      1.00      0.47       303

   micro avg       0.20      0.92      0.33      1161
   macro avg       0.15      0.71      0.23      1161
weighted avg       0.26      0.92      0.40      1161
 samples avg       0.21      0.93      0.34      1161

